# Evaluation

In [5]:
import os
import openai
import sys
sys.path.append('../..')
import utils


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [9]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo-16k", temperature=0, max_tokens=10000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

處理使用者訊息的完整架構

In [13]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # 步驟 1：使用 Moderation API 來檢查輸入是否違反正常使用規則
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        print("步驟 1: 輸入被 Moderation API 檢查出有問題。")
        return "很抱歉， 我們無法處理這訊息"

    if debug: print("步驟 1： 通過訊息適用度檢查。")
    
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
    #print(f"category_and_product_response: {category_and_product_response}")

    # 步驟 2：提取產品列表
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    #print(f"category_and_product_list: {category_and_product_list}")

    if debug: print("步驟 2： 提取產品列表。")

    # 步驟 3：如果找到產品，則查找它們
    product_information = utils.generate_output_string(category_and_product_list)

    if debug: 
        print(f"步驟 3： 查詢產品資訊。")
        #print(f"product_information: {product_information}")

    # 步驟 4：回答使用者問題
    system_message = f"""
    你是一家大型電子商店的客服助理。\
    以友好和樂於助人的口吻回答，並提供簡潔的答案。\
    請確保向使用者提出相關的後續問題。"""
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print(f"步驟 4： 生成回應。")
    all_messages = all_messages + messages[1:]

    # 步驟 5：將回應再次傳送給 Moderation API 做適用度檢查
    response = openai.Moderation.create(input=final_response)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("步驟 5： 回應被 Moderation API 檢查出有問題。")
        return "很抱歉， 我們無法提供這訊息"

    if debug: print("步驟 5： 回應通過適用度檢查。")

    # 步驟 6：詢問模型回應是否足以回答使用者的問題
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    是否回應足以回答使用者的問題？ (Y/N)
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)

    if debug: 
        print(f"步驟 6： 模型完成回應的評估。")
        #print(f"evaluation_response: {evaluation_response}")

    # 步驟 7：如果是（通過評估），則使用此回答；如果不是（無法通過評估），則說將使用者連接到人員

    if "Y" in evaluation_response:   # 使用 "in" 而不是 "==" 來更安全地處理模型輸出變化（例如，"Y." 或 "Yes"）
        if debug: print("步驟 7： 模型通過回應的評估。")
        return final_response, all_messages
    else:
        if debug:
            print(f"步驟 7： 模型未通過回應的評估。")
            #print(f"user_input: {user_input}, final_response: {final_response}\n")
        neg_str = "很抱歉， 我們無法提供這訊息，我們將會將您轉接到服務人員。"
        return neg_str, all_messages

user_input = """
告訴我有關SmartX ProPhone和FotoSnap相機的資訊，
就是那款數位單眼相機的。另外也告訴我你們的電視有哪些。
"""
response,_ = process_user_message(user_input,[])
print(response)

步驟 1： 通過訊息適用度檢查。
步驟 2： 提取產品列表。
步驟 3： 查詢產品資訊。
步驟 4： 生成回應。
步驟 5： 回應通過適用度檢查。
步驟 6： 模型完成回應的評估。
步驟 7： 模型通過回應的評估。
關於SmartX ProPhone和FotoSnap相機的資訊如下：

SmartX ProPhone是一款功能強大並具有先進照相功能的智慧手機。它擁有6.1吋顯示器、128GB儲存空間、12MP雙鏡頭和5G網路。價格為899.99美元。

FotoSnap相機是一款多功能的單眼相機，具有24.2MP感光元件、1080p影片、3吋LCD和可更換鏡頭。價格為599.99美元。

我們的電視產品包括CineView 4K液晶電視、CineView 8K液晶電視和CineView OLED電視。CineView 4K液晶電視擁有55吋顯示器、4K解析度、HDR和智慧電視功能，價格為599.99美元。CineView 8K液晶電視擁有65吋顯示器、8K解析度、HDR和智慧電視功能，價格為2999.99美元。CineView OLED電視擁有55吋顯示器、4K解析度、HDR和智慧電視功能，價格為1499.99美元。

請問您對這些產品有任何特定的問題嗎？
